In [5]:
import os,sys
import pprint 
import numpy as np
import matplotlib.pyplot as plt

C = 1 #Time it takes to write a checkpoint 
R = 1 #Recovery time 


soldict=dict()


def P_success(twindow ,server_age):
    """ P that there is NO failure for twindow time, given server_age """
    #TODO: Empirical CDF
    #Exponential: constant probability
    mttf = 2.0*60
    return max(0, 1.0-(twindow/mttf))

def T_lost(twindow , server_age):
    """ Expected computing time lost due to failure. """
    #Again, not sure if not memoryless, but can assume
    return twindow/2.0 
    


def get_solution(W, nofail, Wdone):
    ret = None
    try:
        ret = soldict[(W, nofail, Wdone)]
    except:
        ret = None
    return ret 


def set_solution(W, nofail, Wdone, best, chunksize):
    soldict[(W, nofail, Wdone)] = (best, chunksize)


def makespan(W, nofail, Wdone, start_time):
    """ Dynamic programming based calculation of E[makespan]"""
    global C 
    global R


    chunksize = 0
    
    if W == 0:
        #set_solution(W, nofail, Wdone, W, 0)
        return (0, 0)

    # if W == 1 and Wdone == 0:
    #     set_solution(W, nofail, Wdone, W, 0)
    #     return (1, 0)

    ret = get_solution(W, nofail, Wdone)
    #print("get soln " + str((W, nofail, Wdone)) + '--> ' + str(ret))
    
    if ret is not None:
        return ret

    best = np.inf
    t = (nofail*start_time) + Wdone

    for i in range(1, W+1):
        #print("i="+str(i))
        #print(str((W, Wdone))+" Spawning success:" + str((W-i, Wdone+i+C)))       
        exp_success = makespan(W-i, nofail, Wdone+i+C, start_time)[0]
        #print(str((W, Wdone))+" Got exp success="+str(exp_success))

        psuc = P_success(i+C, t)
        
        if int(Wdone) != int(R):
            #print(str((W, Wdone))+" Spawning failure:" + str((W, R)))
            exp_fail = makespan(W, 0, R, start_time)[0]
            curr = (psuc*(i+C+exp_success))+((1.0-psuc)*(T_lost(i+C, t) + R + exp_fail))

        else:
            exp_fail = 0
            pfail = 1.0-psuc
            #term was pfail/psuc
            curr = (psuc*(i+C+exp_success))+((pfail/psuc)*(T_lost(i+C, t) + R + exp_fail))


        #print(str((W, Wdone))+" Curr="+str(curr))

        if curr < W:
            #This should not happen, but it still does?
            print("W={}, Wdone={}, i={}, ESUCC={}, EFAIL={}, psuc={}, Curr={}".format(\
                    W, Wdone, i, exp_success, exp_fail, psuc, curr))
            curr = W

        
        if curr < best : #or i==1: #Min quantum 
            #print("FOUND A BEST at i="+str(i))
            best = curr
            chunksize = i

        
    #print(str((W,Wdone))+" Setting soln")
    set_solution(W, nofail, Wdone, best, chunksize)
    
    return get_solution(W, nofail, Wdone)


def dp_driver(W=100):
    makespan(W, 1, 0, 0)

    pprint.pprint(soldict)

                


In [6]:
#P_success(1,2)

In [9]:
makespan(100,1,0,0)

(109.54304127474305, 14)

In [10]:
pprint.pprint(soldict)

{(1, 0, 1): (2.0005649717514125, 1),
 (1, 0, 3): (2.0333427495291905, 1),
 (1, 0, 4): (2.0333427495291905, 1),
 (1, 0, 5): (2.0333427495291905, 1),
 (1, 0, 6): (2.0333427495291905, 1),
 (1, 0, 7): (2.0333427495291905, 1),
 (1, 0, 8): (2.0333427495291905, 1),
 (1, 0, 9): (2.0333427495291905, 1),
 (1, 0, 10): (2.0333427495291905, 1),
 (1, 0, 11): (2.0333427495291905, 1),
 (1, 0, 12): (2.0333427495291905, 1),
 (1, 0, 13): (2.0333427495291905, 1),
 (1, 0, 14): (2.0333427495291905, 1),
 (1, 0, 15): (2.0333427495291905, 1),
 (1, 0, 16): (2.0333427495291905, 1),
 (1, 0, 17): (2.0333427495291905, 1),
 (1, 0, 18): (2.0333427495291905, 1),
 (1, 0, 19): (2.0333427495291905, 1),
 (1, 0, 20): (2.0333427495291905, 1),
 (1, 0, 21): (2.0333427495291905, 1),
 (1, 0, 22): (2.0333427495291905, 1),
 (1, 0, 23): (2.0333427495291905, 1),
 (1, 0, 24): (2.0333427495291905, 1),
 (1, 0, 25): (2.0333427495291905, 1),
 (1, 0, 26): (2.0333427495291905, 1),
 (1, 0, 27): (2.0333427495291905, 1),
 (1, 0, 28): (2.0333

 (4, 0, 188): (5.143493357487923, 4),
 (4, 0, 189): (5.143493357487923, 4),
 (4, 0, 190): (5.143493357487923, 4),
 (4, 0, 191): (5.143493357487923, 4),
 (4, 0, 192): (5.143493357487923, 4),
 (4, 0, 193): (5.143493357487923, 4),
 (4, 1, 97): (5.143493357487923, 4),
 (4, 1, 98): (5.143493357487923, 4),
 (4, 1, 99): (5.143493357487923, 4),
 (4, 1, 100): (5.143493357487923, 4),
 (4, 1, 101): (5.143493357487923, 4),
 (4, 1, 102): (5.143493357487923, 4),
 (4, 1, 103): (5.143493357487923, 4),
 (4, 1, 104): (5.143493357487923, 4),
 (4, 1, 105): (5.143493357487923, 4),
 (4, 1, 106): (5.143493357487923, 4),
 (4, 1, 107): (5.143493357487923, 4),
 (4, 1, 108): (5.143493357487923, 4),
 (4, 1, 109): (5.143493357487923, 4),
 (4, 1, 110): (5.143493357487923, 4),
 (4, 1, 111): (5.143493357487923, 4),
 (4, 1, 112): (5.143493357487923, 4),
 (4, 1, 113): (5.143493357487923, 4),
 (4, 1, 114): (5.143493357487923, 4),
 (4, 1, 115): (5.143493357487923, 4),
 (4, 1, 116): (5.143493357487923, 4),
 (4, 1, 117): (

 (9, 0, 28): (10.4760101010101, 9),
 (9, 0, 29): (10.4760101010101, 9),
 (9, 0, 30): (10.4760101010101, 9),
 (9, 0, 31): (10.4760101010101, 9),
 (9, 0, 32): (10.4760101010101, 9),
 (9, 0, 33): (10.4760101010101, 9),
 (9, 0, 34): (10.4760101010101, 9),
 (9, 0, 35): (10.4760101010101, 9),
 (9, 0, 36): (10.4760101010101, 9),
 (9, 0, 37): (10.4760101010101, 9),
 (9, 0, 38): (10.4760101010101, 9),
 (9, 0, 39): (10.4760101010101, 9),
 (9, 0, 40): (10.4760101010101, 9),
 (9, 0, 41): (10.4760101010101, 9),
 (9, 0, 42): (10.4760101010101, 9),
 (9, 0, 43): (10.4760101010101, 9),
 (9, 0, 44): (10.4760101010101, 9),
 (9, 0, 45): (10.4760101010101, 9),
 (9, 0, 46): (10.4760101010101, 9),
 (9, 0, 47): (10.4760101010101, 9),
 (9, 0, 48): (10.4760101010101, 9),
 (9, 0, 49): (10.4760101010101, 9),
 (9, 0, 50): (10.4760101010101, 9),
 (9, 0, 51): (10.4760101010101, 9),
 (9, 0, 52): (10.4760101010101, 9),
 (9, 0, 53): (10.4760101010101, 9),
 (9, 0, 54): (10.4760101010101, 9),
 (9, 0, 55): (10.47601010101

 (12, 1, 129): (13.758645508826582, 12),
 (12, 1, 130): (13.758645508826582, 12),
 (12, 1, 131): (13.758645508826582, 12),
 (12, 1, 132): (13.758645508826582, 12),
 (12, 1, 133): (13.758645508826582, 12),
 (12, 1, 134): (13.758645508826582, 12),
 (12, 1, 135): (13.758645508826582, 12),
 (12, 1, 136): (13.758645508826582, 12),
 (12, 1, 137): (13.758645508826582, 12),
 (12, 1, 138): (13.758645508826582, 12),
 (12, 1, 139): (13.758645508826582, 12),
 (12, 1, 140): (13.758645508826582, 12),
 (12, 1, 141): (13.758645508826582, 12),
 (12, 1, 142): (13.758645508826582, 12),
 (12, 1, 143): (13.758645508826582, 12),
 (12, 1, 144): (13.758645508826582, 12),
 (12, 1, 145): (13.758645508826582, 12),
 (12, 1, 146): (13.758645508826582, 12),
 (12, 1, 147): (13.758645508826582, 12),
 (12, 1, 148): (13.758645508826582, 12),
 (12, 1, 149): (13.758645508826582, 12),
 (12, 1, 150): (13.758645508826582, 12),
 (12, 1, 151): (13.758645508826582, 12),
 (12, 1, 152): (13.758645508826582, 12),
 (12, 1, 153): (

 (15, 1, 100): (17.10017094017094, 15),
 (15, 1, 101): (17.10017094017094, 15),
 (15, 1, 102): (17.10017094017094, 15),
 (15, 1, 103): (17.10017094017094, 15),
 (15, 1, 104): (17.10017094017094, 15),
 (15, 1, 105): (17.10017094017094, 15),
 (15, 1, 106): (17.10017094017094, 15),
 (15, 1, 107): (17.10017094017094, 15),
 (15, 1, 108): (17.10017094017094, 15),
 (15, 1, 109): (17.10017094017094, 15),
 (15, 1, 110): (17.10017094017094, 15),
 (15, 1, 111): (17.10017094017094, 15),
 (15, 1, 112): (17.10017094017094, 15),
 (15, 1, 113): (17.10017094017094, 15),
 (15, 1, 114): (17.10017094017094, 15),
 (15, 1, 115): (17.10017094017094, 15),
 (15, 1, 116): (17.10017094017094, 15),
 (15, 1, 117): (17.10017094017094, 15),
 (15, 1, 118): (17.10017094017094, 15),
 (15, 1, 119): (17.10017094017094, 15),
 (15, 1, 120): (17.10017094017094, 15),
 (15, 1, 121): (17.10017094017094, 15),
 (15, 1, 122): (17.10017094017094, 15),
 (15, 1, 123): (17.10017094017094, 15),
 (15, 1, 124): (17.10017094017094, 15),


...

 (41, 0, 28): (46.05738932291666, 14),
 (41, 0, 29): (46.05738932291666, 14),
 (41, 0, 30): (46.05738932291666, 14),
 (41, 0, 31): (46.05738932291666, 14),
 (41, 0, 32): (46.05738932291666, 14),
 (41, 0, 33): (46.05738932291666, 14),
 (41, 0, 34): (46.05738932291666, 14),
 (41, 0, 35): (46.05738932291666, 14),
 (41, 0, 36): (46.05738932291666, 14),
 (41, 0, 37): (46.05738932291666, 14),
 (41, 0, 38): (46.05738932291666, 14),
 (41, 0, 39): (46.05738932291666, 14),
 (41, 0, 40): (46.05738932291666, 14),
 (41, 0, 41): (46.05738932291666, 14),
 (41, 0, 42): (46.05738932291666, 14),
 (41, 0, 43): (46.05738932291666, 14),
 (41, 0, 44): (46.05738932291666, 14),
 (41, 0, 45): (46.05738932291666, 14),
 (41, 0, 46): (46.05738932291666, 14),
 (41, 0, 47): (46.05738932291666, 14),
 (41, 0, 48): (46.05738932291666, 14),
 (41, 0, 49): (46.05738932291666, 14),
 (41, 0, 50): (46.05738932291666, 14),
 (41, 0, 51): (46.05738932291666, 14),
 (41, 0, 52): (46.05738932291666, 14),
 (41, 0, 53): (46.0573893

 (46, 0, 86): (51.53033743557474, 15),
 (46, 0, 87): (51.53033743557474, 15),
 (46, 0, 88): (51.53033743557474, 15),
 (46, 0, 89): (51.53033743557474, 15),
 (46, 0, 90): (51.53033743557474, 15),
 (46, 0, 91): (51.53033743557474, 15),
 (46, 0, 92): (51.53033743557474, 15),
 (46, 0, 93): (51.53033743557474, 15),
 (46, 0, 94): (51.53033743557474, 15),
 (46, 0, 95): (51.53033743557474, 15),
 (46, 0, 96): (51.53033743557474, 15),
 (46, 0, 97): (51.53033743557474, 15),
 (46, 0, 98): (51.53033743557474, 15),
 (46, 0, 99): (51.53033743557474, 15),
 (46, 0, 100): (51.53033743557474, 15),
 (46, 0, 101): (51.53033743557474, 15),
 (46, 0, 102): (51.53033743557474, 15),
 (46, 0, 103): (51.53033743557474, 15),
 (46, 0, 104): (51.53033743557474, 15),
 (46, 0, 105): (51.53033743557474, 15),
 (46, 0, 106): (51.53033743557474, 15),
 (46, 0, 107): (51.53033743557474, 15),
 (46, 0, 108): (51.53033743557474, 15),
 (46, 0, 109): (51.53033743557474, 15),
 (46, 1, 55): (51.53033743557474, 15),
 (46, 1, 56): (

 (52, 0, 22): (58.12505861368313, 13),
 (52, 0, 23): (58.12505861368313, 13),
 (52, 0, 24): (58.12505861368313, 13),
 (52, 0, 25): (58.12505861368313, 13),
 (52, 0, 26): (58.12505861368313, 13),
 (52, 0, 27): (58.12505861368313, 13),
 (52, 0, 28): (58.12505861368313, 13),
 (52, 0, 29): (58.12505861368313, 13),
 (52, 0, 30): (58.12505861368313, 13),
 (52, 0, 31): (58.12505861368313, 13),
 (52, 0, 32): (58.12505861368313, 13),
 (52, 0, 33): (58.12505861368313, 13),
 (52, 0, 34): (58.12505861368313, 13),
 (52, 0, 35): (58.12505861368313, 13),
 (52, 0, 36): (58.12505861368313, 13),
 (52, 0, 37): (58.12505861368313, 13),
 (52, 0, 38): (58.12505861368313, 13),
 (52, 0, 39): (58.12505861368313, 13),
 (52, 0, 40): (58.12505861368313, 13),
 (52, 0, 41): (58.12505861368313, 13),
 (52, 0, 42): (58.12505861368313, 13),
 (52, 0, 43): (58.12505861368313, 13),
 (52, 0, 44): (58.12505861368313, 13),
 (52, 0, 45): (58.12505861368313, 13),
 (52, 0, 46): (58.12505861368313, 13),
 (52, 0, 47): (58.1250586

 (66, 0, 3): (73.29645840067653, 13),
 (66, 0, 4): (73.29645840067653, 13),
 (66, 0, 5): (73.29645840067653, 13),
 (66, 0, 6): (73.29645840067653, 13),
 (66, 0, 7): (73.29645840067653, 13),
 (66, 0, 8): (73.29645840067653, 13),
 (66, 0, 9): (73.29645840067653, 13),
 (66, 0, 10): (73.29645840067653, 13),
 (66, 0, 11): (73.29645840067653, 13),
 (66, 0, 12): (73.29645840067653, 13),
 (66, 0, 13): (73.29645840067653, 13),
 (66, 0, 14): (73.29645840067653, 13),
 (66, 0, 15): (73.29645840067653, 13),
 (66, 0, 16): (73.29645840067653, 13),
 (66, 0, 17): (73.29645840067653, 13),
 (66, 0, 18): (73.29645840067653, 13),
 (66, 0, 19): (73.29645840067653, 13),
 (66, 0, 20): (73.29645840067653, 13),
 (66, 0, 21): (73.29645840067653, 13),
 (66, 0, 22): (73.29645840067653, 13),
 (66, 0, 23): (73.29645840067653, 13),
 (66, 0, 24): (73.29645840067653, 13),
 (66, 0, 25): (73.29645840067653, 13),
 (66, 0, 26): (73.29645840067653, 13),
 (66, 0, 27): (73.29645840067653, 13),
 (66, 0, 28): (73.29645840067653

 (85, 1, 25): (93.627107940413, 14),
 (85, 1, 26): (93.627107940413, 14),
 (85, 1, 27): (93.627107940413, 14),
 (85, 1, 28): (93.627107940413, 14),
 (85, 1, 29): (93.627107940413, 14),
 (85, 1, 30): (93.627107940413, 14),
 (86, 0, 1): (86, 12),
 (86, 0, 3): (94.69204717113492, 14),
 (86, 0, 4): (94.69204717113492, 14),
 (86, 0, 5): (94.69204717113492, 14),
 (86, 0, 6): (94.69204717113492, 14),
 (86, 0, 7): (94.69204717113492, 14),
 (86, 0, 8): (94.69204717113492, 14),
 (86, 0, 9): (94.69204717113492, 14),
 (86, 0, 10): (94.69204717113492, 14),
 (86, 0, 11): (94.69204717113492, 14),
 (86, 0, 12): (94.69204717113492, 14),
 (86, 0, 13): (94.69204717113492, 14),
 (86, 0, 14): (94.69204717113492, 14),
 (86, 0, 15): (94.69204717113492, 14),
 (86, 0, 16): (94.69204717113492, 14),
 (86, 0, 17): (94.69204717113492, 14),
 (86, 0, 18): (94.69204717113492, 14),
 (86, 0, 19): (94.69204717113492, 14),
 (86, 0, 20): (94.69204717113492, 14),
 (86, 0, 21): (94.69204717113492, 14),
 (86, 0, 22): (94.692

In [11]:
math.pow(3,4)

NameError: name 'math' is not defined